In [2]:
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [3]:
import os,sys

sys.path.insert(0, "/mnt/f/dev/git/miRExplore/python/")

import time
import spacy
from spacy import displacy

import scispacy

%autoreload 2
from textdb.MiGenRelDB import MiGenRelDB
%autoreload 2
from textdb.SentenceDB import SentenceDB
%autoreload 2
from textmining.MirGeneRelCheck import SentenceRelationClassifier, SentenceRelationChecker

from collections import defaultdict
from natsort import natsorted
from intervaltree import Interval, IntervalTree

In [4]:
nlp = spacy.load('/mnt/f/spacy/en_core_sci_lg-0.2.4/en_core_sci_lg/en_core_sci_lg-0.2.4/')
nlp_ent = spacy.load("/mnt/f/spacy/en_ner_bionlp13cg_md-0.2.4/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.2.4")

In [5]:
sentDBTest = SentenceDB.loadFromFile("/mnt/f/dev/data/pmid_jun2020/mirtex_benchmark/test/", "/mnt/f/dev/data/pmid_jun2020/mirtex_benchmark/development/pmid2sent", redoPmid2Sent=True)
sentDBDev = SentenceDB.loadFromFile("/mnt/f/dev/data/pmid_jun2020/mirtex_benchmark/development/", "/mnt/f/dev/data/pmid_jun2020/mirtex_benchmark/development/pmid2sent", redoPmid2Sent=True)
#mmuDB = MiGenRelDB.loadFromFile("/mnt/f/dev/data/pmid_jun2020/mirtex_benchmark/aggregated_test/mirna_gene.mmu.pmid", ltype="mirna", rtype="gene")
#hsaDB = MiGenRelDB.loadFromFile("/mnt/f/dev/data/pmid_jun2020/mirtex_benchmark/aggregated_test/mirna_gene.hsa.pmid", ltype="mirna", rtype="gene")

Building Sentence DB
Building Sentence DB


In [6]:
relChecker = SentenceRelationChecker(nlp, nlp_ent)
relClassifier = SentenceRelationClassifier()

In [7]:
allSentsTest = sentDBTest.get_all_sentences()
allSentsDev = sentDBDev.get_all_sentences()

allSents = {}
for x in allSentsTest:
    allSents[x] = allSentsTest[x]
for x in allSentsDev:
    allSents[x] = allSentsDev[x]


In [8]:

def analyse(testCase, dist=80):

    doc = nlp(testCase[0])
    entDoc = nlp_ent(testCase[0])
    
    enttree = IntervalTree()
    for ent in entDoc.ents:
        enttree.addi(ent.start_char, ent.end_char, (ent.label_, ent.text))

    
    lWord = doc[testCase[1]]
    rWord = doc[testCase[2]]

    for t in doc:
        
        possTokenEnts = enttree[t.idx: t.idx+len(t.text)]
        ents = set([x.data for x in possTokenEnts])
        
        print(t.idx, t.idx+len(t.text), t.i, t, t.pos_, t.dep_, t.head, t.head.idx, ents)

    print(lWord, rWord)

    if not "mir" in str(lWord).lower() and not "let" in str(lWord).lower():
        tmp = lWord
        lWord = rWord
        rWord = tmp
        
    print(lWord, rWord)
    
    displacy.render(doc, style="dep", options={"compact": True, "distance": dist})
    
    entDoc = nlp_ent(testCase[0])
    
    
    displacy.render(entDoc, style="ent", options={"compact": True, "distance": dist})
    
    return enttree

In [147]:
analyse( (allSents["24319262.2.12"], 9, 10) )

0 2 0 In ADP case article 8 set()
3 7 1 this DET det article 8 set()
8 15 2 article NOUN nmod demonstrate 20 set()
15 16 3 , PUNCT punct demonstrate 20 set()
17 19 4 we PRON nsubj demonstrate 20 set()
20 31 5 demonstrate VERB ROOT demonstrate 20 set()
32 36 6 that SCONJ mark regulated 58 set()
37 44 7 miR-34a PROPN nsubjpass regulated 58 {('GENE_OR_GENE_PRODUCT', 'miR-34a')}
45 48 8 and CCONJ cc miR-34a 37 set()
49 53 9 -145 NUM conj miR-34a 37 set()
54 57 10 are AUX auxpass regulated 58 set()
58 67 11 regulated VERB ccomp demonstrate 20 set()
68 70 12 by ADP case p53 71 set()
71 74 13 p53 NOUN nmod regulated 58 {('GENE_OR_GENE_PRODUCT', 'p53')}
75 78 14 and CCONJ cc regulated 58 set()
79 89 15 negatively ADV advmod regulate 90 set()
90 98 16 regulate VERB conj regulated 58 set()
99 105 17 Twist2 PROPN compound expression 116 {('GENE_OR_GENE_PRODUCT', 'Twist2')}
106 109 18 and CCONJ cc Twist2 99 set()
110 115 19 c-Maf NOUN conj Twist2 99 {('GENE_OR_GENE_PRODUCT', 'c-Maf')}
116 126 20 e

IntervalTree([Interval(37, 44, ('GENE_OR_GENE_PRODUCT', 'miR-34a')), Interval(71, 74, ('GENE_OR_GENE_PRODUCT', 'p53')), Interval(99, 105, ('GENE_OR_GENE_PRODUCT', 'Twist2')), Interval(110, 115, ('GENE_OR_GENE_PRODUCT', 'c-Maf')), Interval(130, 139, ('CELL', 'microglia')), Interval(148, 172, ('CELL', 'RAW macrophage cell line'))])

In [159]:
relChecker = SentenceRelationChecker(nlp, nlp_ent)
relClassifier = SentenceRelationClassifier()

#  (37, 46)	(12, 20)


relRes = relChecker.check_sentence(allSents["22698995.2.4"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (15, 20)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (30, 35)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case let-7
add special case Bach1
let-7 3 Bach1 6
(u"The effects of let-7 miRNA on Bach1 and HMOX1 gene expression in Huh-7 and HepG2 cells were determined by real-time qRT-PCR, Western blot, and luciferase reporter assays.", 3, 6)
geneword Bach1 mirword let-7
Conjuncts [HMOX1, Bach1]
neighbour HMOX1 gene
neighbour HMOX1 gene
Conjunction before fc Bach1
fc add children for Bach1
fc add children for Bach1
Conjuncts [Bach1, HMOX1]
neighbour Bach1 and
neighbour Bach1 and
Conjunction before fc HMOX1
Conjuncts [HepG2, Huh-7]
neighbour HepG2 cells
neighbour HepG2 cells
Conjunction before fc Huh-7
fc add children for Huh-7
fc add children for Huh-7
Conjuncts [Huh-7, HepG2]
neighbour Huh-7 and
neighbour Huh-7 and
Conjunction before fc HepG2
Conjuncts [blot, assays, qRT-PCR]
neighbour blot ,
neighbour blot and
neighbour assays .
neighbour assays None
Conjunction before fc qRT-PCR
fc add children for qRT-PCR
fc add children for qRT-PCR
Conjuncts [qRT-PCR, assays, blot]
neighbour qRT-

{'accept_relation': True,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': True,
  'negation': False,
  'classification': {'regulation_dir': 'NEU',
   'interaction_dir': 'MIR_GENE',
   'passive': True,
   'reg_detect': 'compartment mir gene by',
   'reg_detect_major': 'compartment'}},
 'tagged_sent': 'The effects of <e1>let-7</e1> miRNA on <e2>Bach1</e2> and HMOX1 gene expression in Huh-7 and HepG2 cells were determined by real-time qRT-PCR, Western blot, and luciferase reporter assays.',
 'full_sentence': 'The effects of <e1>let-7</e1> miRNA on <e2>Bach1</e2> and HMOX1 gene expression in Huh-7 and HepG2 cells were determined by real-time qRT-PCR, Western blot, and luciferase reporter assays.',
 'entity1': {'entity_type': 'mirna',
  'entity_type_token': 'e1',
  'entity_location': (15, 20),
  'entity_text': 'let-7'},
 'entity2': {'entity_type': 'gene',
  'entity_type_token': 'e2',
  'entity_location': (30, 35),
  'e

In [137]:
relRes = relChecker.check_sentence(allSents["18185580.2.5"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (184, 191)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (146, 154)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case tenascin
add special case miR-335
tenascin 18 miR-335 24
(u"miR-335 suppresses metastasis and migration through targeting of the progenitor cell transcription factor SOX4 and extracellular matrix component tenascin C. Expression of miR-126 and miR-335 is lost in the majority of primary breast tumours from patients who relapse, and the loss of expression of either microRNA is associated with poor distal metastasis-free survival.", 18, 24)
geneword tenascin mirword miR-335
Conjuncts [lost, associated, suppresses]
neighbour lost in
neighbour lost in
neighbour associated with
neighbour associated with
Conjunction before fc suppresses
fc add children for suppresses
fc add children for suppresses
Conjuncts [migration, metastasis]
neighbour migration through
neighbour migration through
Conjunction before fc metastasis
Conjuncts [metastasis, migration]
neighbour metastasis and
neighbour metastasis and
Conjunction before fc migration
Conjuncts [component, SOX4]
neighbour compon

{'accept_relation': True,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': True,
  'negation': False,
  'classification': {'regulation_dir': 'DOWN',
   'interaction_dir': 'MIR_GENE',
   'passive': True,
   'reg_detect': 'counts between',
   'reg_detect_major': 'counts'}},
 'tagged_sent': 'miR-335 suppresses metastasis and migration through targeting of the progenitor cell transcription factor SOX4 and extracellular matrix component <e2>tenascin</e2> C. Expression of miR-126 and <e1>miR-335</e1> is lost in the majority of primary breast tumours from patients who relapse, and the loss of expression of either microRNA is associated with poor distal metastasis-free survival.',
 'full_sentence': 'miR-335 suppresses metastasis and migration through targeting of the progenitor cell transcription factor SOX4 and extracellular matrix component <e2>tenascin</e2> C. Expression of miR-126 and <e1>miR-335</e1> is lost in the ma

In [27]:
relRes = relChecker.check_sentence(allSents["20299489.2.6"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (103,110)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (32,73)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case mitogen-activated protein kinase kinase 1
add special case miR-34a
1 8 miR-34a 17
(u"Reporter gene assays identified mitogen-activated protein kinase kinase 1 (MEK1) as a direct target of miR-34a and c-fos as a direct target of miR-221/222.", 8, 17)
geneword 1 mirword miR-34a
Conjuncts [c-fos, miR-34a]
neighbour c-fos as
neighbour c-fos as
Conjunction before fc miR-34a
fc add children for miR-34a
Conjuncts [miR-34a, c-fos]
neighbour miR-34a and
neighbour miR-34a and
Conjunction before fc c-fos
MEK1 miR-34a
) as a direct target
Conjunctions
MEK1 [(, )]
miR-34a [of, miR-34a, c-fos]
SDP sent: Reporter gene assays identified mitogen-activated protein kinase kinase 1 (MEK1) as a direct target of miR-34a and c-fos as a direct target of miR-221/222.
SDP     : [(1, 'NUM', 'nummod', False), (kinase, 'NOUN', 'dobj', False), (identified, 'VERB', 'ROOT', False), (target, 'NOUN', 'nmod', False), (miR-34a, 'PROPN', 'nmod', False)]
[c-fos]
0 [Reporter, gene, assays, identified, mitog

{'accept_relation': True,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': False,
  'negation': False,
  'classification': {'regulation_dir': 'NEU',
   'interaction_dir': 'GENE_MIR',
   'passive': False,
   'reg_detect': 'compartment gene mir',
   'reg_detect_major': 'compartment'}},
 'tagged_sent': 'Reporter gene assays identified <e2>mitogen-activated protein kinase kinase 1</e2> (MEK1) as a direct target of <e1>miR-34a</e1> and c-fos as a direct target of miR-221/222.',
 'full_sentence': 'Reporter gene assays identified <e2>mitogen-activated protein kinase kinase 1</e2> (MEK1) as a direct target of <e1>miR-34a</e1> and c-fos as a direct target of miR-221/222.',
 'entity1': {'entity_type': 'gene',
  'entity_type_token': 'e2',
  'entity_location': (32, 73),
  'entity_text': 'mitogen-activated protein kinase kinase 1'},
 'entity2': {'entity_type': 'mirna',
  'entity_type_token': 'e1',
  'entity_location': (103, 110

In [10]:
relRes = relChecker.check_sentence(allSents["23041385.2.11"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (29,35)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (0,10)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case TGF-betaR2
add special case miR-21
TGF-betaR2 0 miR-21 5
(u"TGF-betaR2 mRNA, a validated miR-21 target, showed the highest expression in the leukocytes from a subset of the octogenarians.", 0, 5)
geneword TGF-betaR2 mirword miR-21
Conjunctions
target [a, target]
SDP sent: TGF-betaR2 mRNA, a validated miR-21 target, showed the highest expression in the leukocytes from a subset of the octogenarians.
SDP     : [(TGF-betaR2, 'PROPN', 'amod', False), (mRNA, 'NOUN', 'nsubj', False), (target, 'NOUN', 'appos', False), (miR-21, 'PROPN', 'compound', False)]
0 [TGF-betaR2, mRNA, ,, a, validated, miR-21, target, ,, showed, the, highest, expression, in, the, leukocytes, from, a, subset, of, the, octogenarians] 21 0 TGF-betaR2 mRNA, a validated miR-21 target, showed the highest expression in the leukocytes from a subset of the octogenarians.
Recognized overlapping entities: {Interval(0, 10, ('GENE_OR_GENE_PRODUCT', 'TGF-betaR2'))}
[Interval(0, 10, ('GENE_OR_GENE_PRODUCT', 'TGF-betaR

{'accept_relation': True,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': False,
  'negation': False,
  'classification': {'regulation_dir': 'NEU',
   'interaction_dir': 'MIR_GENE',
   'passive': False,
   'reg_detect': 'between gene mir',
   'reg_detect_major': 'between'}},
 'tagged_sent': '<e2>TGF-betaR2</e2> mRNA, a validated <e1>miR-21</e1> target, showed the highest expression in the leukocytes from a subset of the octogenarians.',
 'full_sentence': '<e2>TGF-betaR2</e2> mRNA, a validated <e1>miR-21</e1> target, showed the highest expression in the leukocytes from a subset of the octogenarians.',
 'entity1': {'entity_type': 'gene',
  'entity_type_token': 'e2',
  'entity_location': (0, 10),
  'entity_text': 'TGF-betaR2'},
 'entity2': {'entity_type': 'mirna',
  'entity_type_token': 'e1',
  'entity_location': (29, 35),
  'entity_text': 'miR-21'}}

In [11]:
relRes = relChecker.check_sentence(allSents["23113351.2.10"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (48,57)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (149,153)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case miRNA-29a
add special case E2F1
miRNA-29a 9 E2F1 24
(u"In summary, data obtained highlight the role of miRNA-29a in the regulation of osteoblastic cell apoptosis by silencing Bcl-2 and Mcl-1 and inducing E2F1 and E2F3 expression.", 9, 24)
geneword E2F1 mirword miRNA-29a
Conjuncts [inducing, silencing]
neighbour inducing E2F1
neighbour inducing E2F1
Conjunction before fc silencing
fc add children for silencing
Conjuncts [Mcl-1, Bcl-2]
neighbour Mcl-1 and
neighbour Mcl-1 and
Conjunction before fc Bcl-2
Conjuncts [Bcl-2, Mcl-1]
neighbour Bcl-2 and
neighbour Bcl-2 and
Conjunction before fc Mcl-1
Conjuncts [silencing, inducing]
neighbour silencing Bcl-2
neighbour silencing Bcl-2
Conjunction before fc inducing
fc add children for inducing
Conjuncts [E2F3, E2F1]
neighbour E2F3 expression
neighbour E2F3 expression
Conjunction before fc E2F1
Conjuncts [E2F1, E2F3]
neighbour E2F1 and
neighbour E2F1 and
Conjunction before fc E2F3
Conjunctions
silencing [silencing, Bcl-2, Mcl-1, i

{'accept_relation': True,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': False,
  'negation': False,
  'classification': {'regulation_dir': 'UP',
   'interaction_dir': 'MIR_GENE',
   'passive': False,
   'reg_detect': 'counts between',
   'reg_detect_major': 'counts'}},
 'tagged_sent': 'In summary, data obtained highlight the role of <e1>miRNA-29a</e1> in the regulation of osteoblastic cell apoptosis by silencing Bcl-2 and Mcl-1 and inducing <e2>E2F1</e2> and E2F3 expression.',
 'full_sentence': 'In summary, data obtained highlight the role of <e1>miRNA-29a</e1> in the regulation of osteoblastic cell apoptosis by silencing Bcl-2 and Mcl-1 and inducing <e2>E2F1</e2> and E2F3 expression.',
 'entity1': {'entity_type': 'mirna',
  'entity_type_token': 'e1',
  'entity_location': (48, 57),
  'entity_text': 'miRNA-29a'},
 'entity2': {'entity_type': 'gene',
  'entity_type_token': 'e2',
  'entity_location': (149, 153),
  '

In [12]:
relRes = relChecker.check_sentence(allSents["23190608.2.4"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (102,109)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (14,17)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case Sp1
add special case miR-29b
Sp1 2 miR-29b 17
(u"We identified Sp1, a transcription factor endowed with oncogenic activity, as a negative regulator of miR-29b expression in MM cells.", 2, 17)
geneword Sp1 mirword miR-29b
Conjunctions
factor [a, activity]
SDP sent: We identified Sp1, a transcription factor endowed with oncogenic activity, as a negative regulator of miR-29b expression in MM cells.
SDP     : [(Sp1, 'PROPN', 'dobj', False), (identified, 'VERB', 'ROOT', False), (regulator, 'NOUN', 'nmod', False), (expression, 'NOUN', 'nmod', False), (miR-29b, 'PROPN', 'compound', False)]
0 [We, identified, Sp1, ,, a, transcription, factor, endowed, with, oncogenic, activity, ,, as, a, negative, regulator, of, miR-29b, expression, in, MM, cells] 22 0 We identified Sp1, a transcription factor endowed with oncogenic activity, as a negative regulator of miR-29b expression in MM cells.
Recognized overlapping entities: {Interval(14, 17, ('GENE_OR_GENE_PRODUCT', 'Sp1'))}
[Interval

{'accept_relation': True,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': False,
  'negation': False,
  'classification': {'regulation_dir': 'NEU',
   'interaction_dir': 'GENE_MIR',
   'passive': False,
   'reg_detect': 'compartment gene mir',
   'reg_detect_major': 'compartment'}},
 'tagged_sent': 'We identified <e2>Sp1</e2>, a transcription factor endowed with oncogenic activity, as a negative regulator of <e1>miR-29b</e1> expression in MM cells.',
 'full_sentence': 'We identified <e2>Sp1</e2>, a transcription factor endowed with oncogenic activity, as a negative regulator of <e1>miR-29b</e1> expression in MM cells.',
 'entity1': {'entity_type': 'gene',
  'entity_type_token': 'e2',
  'entity_location': (14, 17),
  'entity_text': 'Sp1'},
 'entity2': {'entity_type': 'mirna',
  'entity_type_token': 'e1',
  'entity_location': (102, 109),
  'entity_text': 'miR-29b'}}

In [13]:
relRes = relChecker.check_sentence(allSents["23643257.2.2"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (14,23)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (45,79)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case miRNA-424
add special case mitogen-activated protein kinase 1
miRNA-424 3 1 12
(u"The levels of miRNA-424 and its target gene, mitogen-activated protein kinase 1 (MEK1), and fibroblast growth factor receptor 1 (FGFR1) were examined.", 3, 12)
geneword 1 mirword miRNA-424
Conjuncts [receptor, levels]
neighbour receptor 1
neighbour receptor 1
Conjunction before fc levels
Conjuncts [gene, miRNA-424]
neighbour gene ,
neighbour gene mitogen-activated
Conjunction before fc miRNA-424
fc add children for miRNA-424
Conjuncts [miRNA-424, gene]
neighbour miRNA-424 and
neighbour miRNA-424 and
Conjunction before fc gene
fc add children for gene
fc add children for gene
Conjuncts [levels, receptor]
neighbour levels of
neighbour levels of
Conjunction before fc receptor
fc add children for receptor
fc add children for receptor
fc add children for receptor
fc add children for receptor
fc add children for receptor
levels miRNA-424

miRNA-424 FGFR1
), and fibroblast growth factor receptor

{'accept_relation': True,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': True,
  'negation': False,
  'classification': {'regulation_dir': 'NEU',
   'interaction_dir': 'MIR_GENE',
   'passive': False,
   'reg_detect': 'between mir gene',
   'reg_detect_major': 'between'}},
 'tagged_sent': 'The levels of <e1>miRNA-424</e1> and its target gene, <e2>mitogen-activated protein kinase 1</e2> (MEK1), and fibroblast growth factor receptor 1 (FGFR1) were examined.',
 'full_sentence': 'The levels of <e1>miRNA-424</e1> and its target gene, <e2>mitogen-activated protein kinase 1</e2> (MEK1), and fibroblast growth factor receptor 1 (FGFR1) were examined.',
 'entity1': {'entity_type': 'mirna',
  'entity_type_token': 'e1',
  'entity_location': (14, 23),
  'entity_text': 'miRNA-424'},
 'entity2': {'entity_type': 'gene',
  'entity_type_token': 'e2',
  'entity_location': (45, 79),
  'entity_text': 'mitogen-activated protein kinase

In [14]:
relRes = relChecker.check_sentence(allSents["23759586.2.8"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (17,25)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (119,124)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case miR-125b
add special case SIRT1
miR-125b 3 SIRT1 15
(u"Use of antisense miR-125b transcripts enhanced expression of pro-apoptotic p53, repressed expression of anti-apoptotic SIRT1 and, importantly, significantly enhanced dexamethasone-induced cell death responses in MM.", 3, 15)
geneword SIRT1 mirword miR-125b
Conjuncts [repressed, enhanced, enhanced]
neighbour repressed expression
neighbour repressed expression
neighbour enhanced dexamethasone-induced
neighbour enhanced dexamethasone-induced
Conjunction before fc enhanced
fc add children for enhanced
Conjuncts [enhanced, enhanced, repressed]
neighbour enhanced expression
neighbour enhanced expression
neighbour enhanced dexamethasone-induced
neighbour enhanced dexamethasone-induced
Conjunction before fc repressed
fc add children for repressed
Conjuncts [enhanced, repressed, enhanced]
neighbour enhanced expression
neighbour enhanced expression
neighbour repressed expression
neighbour repressed expression
Conjunction bef

{'accept_relation': True,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': False,
  'negation': False,
  'classification': {'regulation_dir': 'DOWN',
   'interaction_dir': 'MIR_GENE',
   'passive': False,
   'reg_detect': 'counts between equal',
   'reg_detect_major': 'counts'}},
 'tagged_sent': 'Use of antisense <e1>miR-125b</e1> transcripts enhanced expression of pro-apoptotic p53, repressed expression of anti-apoptotic <e2>SIRT1</e2> and, importantly, significantly enhanced dexamethasone-induced cell death responses in MM.',
 'full_sentence': 'Use of antisense <e1>miR-125b</e1> transcripts enhanced expression of pro-apoptotic p53, repressed expression of anti-apoptotic <e2>SIRT1</e2> and, importantly, significantly enhanced dexamethasone-induced cell death responses in MM.',
 'entity1': {'entity_type': 'mirna',
  'entity_type_token': 'e1',
  'entity_location': (17, 25),
  'entity_text': 'miR-125b'},
 'entity2': 

In [15]:
relRes = relChecker.check_sentence(allSents["24006456.2.4"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (0,7)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (120,146)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case miR-29b
add special case leukemia inhibitory factor
miR-29b 0 factor 20
(u"miR-29b targeted a cadre of proteins involved in fibrosis, including multiple collagens, matrix metalloproteinases, and leukemia inhibitory factor, insulin-like growth factor 1, and pentraxin 3, 3 predicted targets of miR-29b.", 0, 20)
geneword factor mirword miR-29b
Conjuncts [metalloproteinases, factor, factor, pentraxin, targets, collagens]
neighbour metalloproteinases ,
neighbour metalloproteinases and
neighbour factor ,
neighbour factor insulin-like
neighbour factor 1
neighbour factor 1
neighbour pentraxin 3
neighbour pentraxin 3
neighbour targets of
neighbour targets of
Conjunction before fc collagens
fc add children for collagens
fc add children for collagens
Conjuncts [collagens, factor, factor, pentraxin, targets, metalloproteinases]
neighbour collagens ,
neighbour collagens matrix
neighbour factor ,
neighbour factor insulin-like
neighbour factor 1
neighbour factor 1
neighbour pentraxin

{'accept_relation': True,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': False,
  'negation': False,
  'classification': {'regulation_dir': 'NEU',
   'interaction_dir': 'MIR_GENE',
   'passive': False,
   'reg_detect': 'compartment mir gene',
   'reg_detect_major': 'compartment'}},
 'tagged_sent': '<e1>miR-29b</e1> targeted a cadre of proteins involved in fibrosis, including multiple collagens, matrix metalloproteinases, and <e2>leukemia inhibitory factor</e2>, insulin-like growth factor 1, and pentraxin 3, 3 predicted targets of miR-29b.',
 'full_sentence': '<e1>miR-29b</e1> targeted a cadre of proteins involved in fibrosis, including multiple collagens, matrix metalloproteinases, and <e2>leukemia inhibitory factor</e2>, insulin-like growth factor 1, and pentraxin 3, 3 predicted targets of miR-29b.',
 'entity1': {'entity_type': 'mirna',
  'entity_type_token': 'e1',
  'entity_location': (0, 7),
  'entity_text': '

In [36]:
analyse( (allSents["24219008.2.7"], 1,2) )

0 10 0 Hsa-miR-24 NOUN compound 5p 13 {('GENE_OR_GENE_PRODUCT', 'Hsa-miR-24-2-5p')}
10 11 1 - PUNCT punct Hsa-miR-24 0 {('GENE_OR_GENE_PRODUCT', 'Hsa-miR-24-2-5p')}
11 12 2 2 NUM nummod 5p 13 {('GENE_OR_GENE_PRODUCT', 'Hsa-miR-24-2-5p')}
12 13 3 - PUNCT punct 5p 13 {('GENE_OR_GENE_PRODUCT', 'Hsa-miR-24-2-5p')}
13 15 4 5p NOUN nsubj likely 20 {('GENE_OR_GENE_PRODUCT', 'Hsa-miR-24-2-5p')}
16 19 5 was AUX cop likely 20 set()
20 26 6 likely ADJ ROOT likely 20 set()
27 29 7 to PART mark responsible 33 set()
30 32 8 be AUX cop responsible 33 set()
33 44 9 responsible ADJ xcomp likely 20 set()
45 48 10 for ADP case MYC 49 set()
49 52 11 MYC PROPN nmod responsible 33 {('GENE_OR_GENE_PRODUCT', 'MYC')}
53 56 12 and CCONJ cc MYC 49 set()
57 62 13 KCNJ2 NOUN compound genes 63 {('GENE_OR_GENE_PRODUCT', 'KCNJ2')}
63 68 14 genes NOUN conj MYC 49 set()
69 72 15 and CCONJ cc MYC 49 set()
73 87 16 hsa-miR-27a-5p NOUN conj MYC 49 {('GENE_OR_GENE_PRODUCT', 'hsa-miR-27a-5p')}
88 91 17 for ADP case ST3GAL6 

IntervalTree([Interval(0, 15, ('GENE_OR_GENE_PRODUCT', 'Hsa-miR-24-2-5p')), Interval(49, 52, ('GENE_OR_GENE_PRODUCT', 'MYC')), Interval(57, 62, ('GENE_OR_GENE_PRODUCT', 'KCNJ2')), Interval(73, 87, ('GENE_OR_GENE_PRODUCT', 'hsa-miR-27a-5p')), Interval(92, 99, ('GENE_OR_GENE_PRODUCT', 'ST3GAL6'))])

In [37]:
relRes = relChecker.check_sentence(allSents["24219008.2.7"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (0,12)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (92,99)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case Hsa-miR-24-2
add special case ST3GAL6
2 2 ST3GAL6 18
(u"Hsa-miR-24-2-5p was likely to be responsible for MYC and KCNJ2 genes and hsa-miR-27a-5p for ST3GAL6.", 2, 18)
geneword ST3GAL6 mirword 2
Conjuncts [genes, hsa-miR-27a-5p, MYC]
neighbour genes and
neighbour genes and
neighbour hsa-miR-27a-5p for
neighbour hsa-miR-27a-5p for
Conjunction before fc MYC
fc add children for MYC
Conjuncts [MYC, hsa-miR-27a-5p, genes]
neighbour MYC and
neighbour MYC and
neighbour hsa-miR-27a-5p for
neighbour hsa-miR-27a-5p for
Conjunction before fc genes
fc add children for genes
Conjuncts [MYC, genes, hsa-miR-27a-5p]
neighbour MYC and
neighbour MYC and
neighbour genes and
neighbour genes and
Conjunction before fc hsa-miR-27a-5p
Conjunctions
MYC [for, MYC, KCNJ2, genes, hsa-miR-27a-5p]
SDP sent: Hsa-miR-24-2-5p was likely to be responsible for MYC and KCNJ2 genes and hsa-miR-27a-5p for ST3GAL6.
SDP     : [(2, 'NUM', 'nummod', False), (5p, 'NOUN', 'nsubj', False), (likely, 'ADJ', 'ROOT', F

{'accept_relation': True,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': False,
  'negation': False,
  'classification': {'regulation_dir': 'NEU',
   'interaction_dir': 'MIR_GENE',
   'passive': False,
   'reg_detect': 'return',
   'reg_detect_major': 'return'}},
 'tagged_sent': '<e1>Hsa-miR-24-2</e1>-5p was likely to be responsible for MYC and KCNJ2 genes and hsa-miR-27a-5p for <e2>ST3GAL6</e2>.',
 'full_sentence': '<e1>Hsa-miR-24-2</e1>-5p was likely to be responsible for MYC and KCNJ2 genes and hsa-miR-27a-5p for <e2>ST3GAL6</e2>.',
 'entity1': {'entity_type': 'mirna',
  'entity_type_token': 'e1',
  'entity_location': (0, 12),
  'entity_text': 'Hsa-miR-24-2'},
 'entity2': {'entity_type': 'gene',
  'entity_type_token': 'e2',
  'entity_location': (92, 99),
  'entity_text': 'ST3GAL6'}}

In [17]:
relRes = relChecker.check_sentence(allSents["22767443.2.3"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (85,91)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (148,155)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case miR-33
add special case SREBP-2
miR-33 16 SREBP-2 28
(u"We show that two of these transporters, ABCB11 and ATP8B1, are functional targets of miR-33, a micro-RNA that is expressed from within an intron of SREBP-2.", 16, 28)
geneword SREBP-2 mirword miR-33
Conjuncts [ABCB11, ATP8B1, two]
neighbour ABCB11 and
neighbour ABCB11 and
neighbour ATP8B1 ,
neighbour ATP8B1 are
Conjunction before fc two
Conjuncts [two, ATP8B1, ABCB11]
neighbour two of
neighbour two of
neighbour ATP8B1 ,
neighbour ATP8B1 are
Conjunction before fc ABCB11
Conjuncts [two, ABCB11, ATP8B1]
neighbour two of
neighbour two of
neighbour ABCB11 and
neighbour ABCB11 and
Conjunction before fc ATP8B1
two micro-RNA
ATP8B1, are functional targets of miR-33,
Conjunctions
two [two, ABCB11, ATP8B1]
micro-RNA [a, SREBP-2]
SDP sent: We show that two of these transporters, ABCB11 and ATP8B1, are functional targets of miR-33, a micro-RNA that is expressed from within an intron of SREBP-2.
SDP     : [(miR-33, 'PROPN', 'n

{'accept_relation': False,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': False,
  'entity': True,
  'passive': False,
  'negation': False,
  'classification': {'regulation_dir': 'NEU',
   'interaction_dir': 'MIR_GENE',
   'passive': False,
   'reg_detect': 'compartment mir gene',
   'reg_detect_major': 'compartment'}},
 'tagged_sent': 'We show that two of these transporters, ABCB11 and ATP8B1, are functional targets of <e1>miR-33</e1>, a micro-RNA that is expressed from within an intron of <e2>SREBP-2</e2>.',
 'full_sentence': 'We show that two of these transporters, ABCB11 and ATP8B1, are functional targets of <e1>miR-33</e1>, a micro-RNA that is expressed from within an intron of <e2>SREBP-2</e2>.',
 'entity1': {'entity_type': 'mirna',
  'entity_type_token': 'e1',
  'entity_location': (85, 91),
  'entity_text': 'miR-33'},
 'entity2': {'entity_type': 'gene',
  'entity_type_token': 'e2',
  'entity_location': (148, 155),
  'entity_text': 'SREBP-2'}}

In [18]:
relRes = relChecker.check_sentence(allSents["22431589.2.2"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (27,33)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (165,168)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case miR-25
add special case MYC
miR-25 7 MYC 27
(u"In this study, two miRNAs, miR-25 and -32, are identified as p53-repressed miRNAs by p53-dependent negative regulation of their transcriptional regulators, E2F1 and MYC.", 7, 27)
geneword MYC mirword miR-25
Conjuncts [-32, miR-25]
neighbour -32 ,
neighbour -32 are
Conjunction before fc miR-25
Conjuncts [miR-25, -32]
neighbour miR-25 and
neighbour miR-25 and
Conjunction before fc -32
Conjuncts [MYC, E2F1]
neighbour MYC .
neighbour MYC None
Conjunction before fc E2F1
Conjuncts [E2F1, MYC]
neighbour E2F1 and
neighbour E2F1 and
Conjunction before fc MYC
miR-25 E2F1
-32, are identified as p53-repressed miRNAs by p53-dependent negative regulation of their transcriptional regulators,
Conjunctions
miR-25 [miR-25, -32]
E2F1 [E2F1, MYC]
SDP sent: In this study, two miRNAs, miR-25 and -32, are identified as p53-repressed miRNAs by p53-dependent negative regulation of their transcriptional regulators, E2F1 and MYC.
SDP     : [(miR-25,

{'accept_relation': True,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': True,
  'negation': False,
  'classification': {'regulation_dir': 'UP',
   'interaction_dir': 'MIR_GENE',
   'passive': False,
   'reg_detect': 'between mir gene reg corr',
   'reg_detect_major': 'between'}},
 'tagged_sent': 'In this study, two miRNAs, <e1>miR-25</e1> and -32, are identified as p53-repressed miRNAs by p53-dependent negative regulation of their transcriptional regulators, E2F1 and <e2>MYC</e2>.',
 'full_sentence': 'In this study, two miRNAs, <e1>miR-25</e1> and -32, are identified as p53-repressed miRNAs by p53-dependent negative regulation of their transcriptional regulators, E2F1 and <e2>MYC</e2>.',
 'entity1': {'entity_type': 'mirna',
  'entity_type_token': 'e1',
  'entity_location': (27, 33),
  'entity_text': 'miR-25'},
 'entity2': {'entity_type': 'gene',
  'entity_type_token': 'e2',
  'entity_location': (165, 168),
  'en

In [19]:
relRes = relChecker.check_sentence(allSents["22012398.2.4"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (17,29)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (151,157)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case microRNA-33b
add special case SREBF2
microRNA-33b 2 SREBF2 19
(u"MicroRNA-33a and microRNA-33b (miR-33a/b) are intronic miRNAs whose encoding regions are embedded in the sterol-response-element-binding protein genes SREBF2 and SREBF1 (refs 3-5), respectively.", 2, 19)
geneword SREBF2 mirword microRNA-33b
Conjuncts [microRNA-33b, MicroRNA-33a]
neighbour microRNA-33b (
neighbour microRNA-33b miR-33a/b
Conjunction before fc MicroRNA-33a
Conjuncts [MicroRNA-33a, microRNA-33b]
neighbour MicroRNA-33a and
neighbour MicroRNA-33a and
Conjunction before fc microRNA-33b
fc add children for microRNA-33b
Conjuncts [SREBF1, SREBF2]
neighbour SREBF1 (
neighbour SREBF1 refs
Conjunction before fc SREBF2
Conjuncts [SREBF2, SREBF1]
neighbour SREBF2 and
neighbour SREBF2 and
Conjunction before fc SREBF1
MicroRNA-33a miR-33a/b

miR-33a/b SREBF2
) are intronic miRNAs whose encoding regions are embedded in the sterol-response-element-binding protein genes
Conjunctions
MicroRNA-33a [MicroRNA-3

{'accept_relation': False,
 'check_results': {'conj': True,
  'sdp': False,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': False,
  'negation': False,
  'classification': {'regulation_dir': 'NEU',
   'interaction_dir': 'MIR_GENE',
   'passive': False,
   'reg_detect': 'return',
   'reg_detect_major': 'return'}},
 'tagged_sent': 'MicroRNA-33a and <e1>microRNA-33b</e1> (miR-33a/b) are intronic miRNAs whose encoding regions are embedded in the sterol-response-element-binding protein genes <e2>SREBF2</e2> and SREBF1 (refs 3-5), respectively.',
 'full_sentence': 'MicroRNA-33a and <e1>microRNA-33b</e1> (miR-33a/b) are intronic miRNAs whose encoding regions are embedded in the sterol-response-element-binding protein genes <e2>SREBF2</e2> and SREBF1 (refs 3-5), respectively.',
 'entity1': {'entity_type': 'mirna',
  'entity_type_token': 'e1',
  'entity_location': (17, 29),
  'entity_text': 'microRNA-33b'},
 'entity2': {'entity_type': 'gene',
  'entity_type_token': 'e2'

In [20]:
relRes = relChecker.check_sentence(allSents["20637509.2.9"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (12,20)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (121,131)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case miR-106b
add special case CDKN1A/p21
miR-106b 0 CDKN1A/p21 16
(u"miR-106b and miR-25 were previously shown to modulate the TGF-beta signaling pathway through their action on CDKN1A/p21 and BCL2L11/Bim.", 0, 16)
geneword CDKN1A/p21 mirword miR-106b
Conjuncts [miR-25, miR-106b]
neighbour miR-25 were
neighbour miR-25 were
Conjunction before fc miR-106b
Conjuncts [miR-106b, miR-25]
neighbour miR-106b and
neighbour miR-106b and
Conjunction before fc miR-25
Conjuncts [BCL2L11/Bim, CDKN1A/p21]
neighbour BCL2L11/Bim .
neighbour BCL2L11/Bim None
Conjunction before fc CDKN1A/p21
fc add children for CDKN1A/p21
Conjuncts [CDKN1A/p21, BCL2L11/Bim]
neighbour CDKN1A/p21 and
neighbour CDKN1A/p21 and
Conjunction before fc BCL2L11/Bim
miR-106b CDKN1A/p21
miR-25 were previously shown to modulate the TGF-beta signaling pathway through their action
Conjunctions
miR-106b [miR-106b, miR-25]
CDKN1A/p21 [on, CDKN1A/p21, BCL2L11/Bim]
SDP sent: miR-106b and miR-25 were previously shown to modula

{'accept_relation': False,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': False,
  'passive': True,
  'negation': False,
  'classification': {'regulation_dir': 'NEU',
   'interaction_dir': 'MIR_GENE',
   'passive': True,
   'reg_detect': 'return',
   'reg_detect_major': 'return'}},
 'tagged_sent': '<e1>miR-106b</e1> and miR-25 were previously shown to modulate the TGF-beta signaling pathway through their action on <e2>CDKN1A/p21</e2> and BCL2L11/Bim.',
 'full_sentence': 'CONCLUSION; <e1>miR-106b</e1> and miR-25 were previously shown to modulate the TGF-beta signaling pathway through their action on <e2>CDKN1A/p21</e2> and BCL2L11/Bim.',
 'entity1': {'entity_type': 'mirna',
  'entity_type_token': 'e1',
  'entity_location': (0, 8),
  'entity_text': 'miR-106b'},
 'entity2': {'entity_type': 'gene',
  'entity_type_token': 'e2',
  'entity_location': (109, 119),
  'entity_text': 'CDKN1A/p21'}}

In [21]:
relRes = relChecker.check_sentence(allSents["20566686.2.2"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (82,87)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (104,108)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case miR-1
add special case mTOR
miR-1 15 mTOR 19
(u"In this study, we report that expression of the muscle-specific micro-RNA (miRNA) miR-1 is regulated by mTOR both in differentiating myoblasts and in mouse regenerating skeletal muscle.", 15, 19)
geneword mTOR mirword miR-1
Conjuncts [mouse, myoblasts]
neighbour mouse regenerating
neighbour mouse regenerating
Conjunction before fc myoblasts
fc add children for myoblasts
fc add children for myoblasts
Conjuncts [myoblasts, mouse]
neighbour myoblasts and
neighbour myoblasts and
Conjunction before fc mouse
fc add children for mouse
fc add children for mouse
miRNA myoblasts
) miR-1 is regulated by mTOR both
Conjunctions
miRNA [(, )]
myoblasts [in, differentiating, myoblasts, in, mouse, regenerating, skeletal, muscle]
SDP sent: In this study, we report that expression of the muscle-specific micro-RNA (miRNA) miR-1 is regulated by mTOR both in differentiating myoblasts and in mouse regenerating skeletal muscle.
SDP     : [(miR-1

{'accept_relation': True,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': True,
  'negation': False,
  'classification': {'regulation_dir': 'NEU',
   'interaction_dir': 'GENE_MIR',
   'passive': False,
   'reg_detect': 'between mir gene rb',
   'reg_detect_major': 'between'}},
 'tagged_sent': 'In this study, we report that expression of the muscle-specific micro-RNA (miRNA) <e1>miR-1</e1> is regulated by <e2>mTOR</e2> both in differentiating myoblasts and in mouse regenerating skeletal muscle.',
 'full_sentence': 'In this study, we report that expression of the muscle-specific micro-RNA (miRNA) <e1>miR-1</e1> is regulated by <e2>mTOR</e2> both in differentiating myoblasts and in mouse regenerating skeletal muscle.',
 'entity1': {'entity_type': 'mirna',
  'entity_type_token': 'e1',
  'entity_location': (82, 87),
  'entity_text': 'miR-1'},
 'entity2': {'entity_type': 'gene',
  'entity_type_token': 'e2',
  'entity_lo

In [38]:
relRes = relChecker.check_sentence(allSents["16929162.2.8"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (209,217)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (157,161)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case TCF8
add special case miR-200c
TCF8 27 miR-200c 37
(u"Expression analysis by qRT-PCR of these targets and an additional ten mRNAs (selected using the prediction program ranking alone) revealed four mRNAs, BIN1, TCF8, RND3 and LHFP with an inverse relationship to miR-200c.", 27, 37)
geneword TCF8 mirword miR-200c
Conjuncts [mRNAs, qRT-PCR]
neighbour mRNAs (
neighbour mRNAs selected
Conjunction before fc qRT-PCR
fc add children for qRT-PCR
Conjuncts [qRT-PCR, mRNAs]
neighbour qRT-PCR of
neighbour qRT-PCR of
Conjunction before fc mRNAs
fc add children for mRNAs
fc add children for mRNAs
fc add children for mRNAs
Conjuncts [TCF8, mRNAs]
neighbour TCF8 ,
neighbour TCF8 RND3
Conjunction before fc mRNAs
fc add children for mRNAs
fc add children for mRNAs
fc add children for mRNAs
Conjuncts [mRNAs, TCF8]
neighbour mRNAs ,
neighbour mRNAs BIN1
Conjunction before fc TCF8
Conjuncts [LHFP, RND3]
neighbour LHFP with
neighbour LHFP with
Conjunction before fc RND3
Conjuncts [RND3, LH

{'accept_relation': True,
 'check_results': {'conj': True,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': False,
  'negation': False,
  'classification': {'regulation_dir': 'DOWN',
   'interaction_dir': 'MIR_GENE',
   'passive': False,
   'reg_detect': 'between gene mir neg corr',
   'reg_detect_major': 'between'}},
 'tagged_sent': 'Expression analysis by qRT-PCR of these targets and an additional ten mRNAs (selected using the prediction program ranking alone) revealed four mRNAs, BIN1, <e2>TCF8</e2>, RND3 and LHFP with an inverse relationship to <e1>miR-200c</e1>.',
 'full_sentence': 'Expression analysis by qRT-PCR of these targets and an additional ten mRNAs (selected using the prediction program ranking alone) revealed four mRNAs, BIN1, <e2>TCF8</e2>, RND3 and LHFP with an inverse relationship to <e1>miR-200c</e1>.',
 'entity1': {'entity_type': 'gene',
  'entity_type_token': 'e2',
  'entity_location': (157, 161),
  'entity_text': 'TCF8'},
 'e

In [23]:
relRes = relChecker.check_sentence(allSents["21932404.2.9"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (141,149)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (99,104)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case MMP-2
add special case miR-181b
MMP-2 12 miR-181b 21
(u"Furthermore, treatment of SMCCs with SCF+GM-CSF significantly increased matrix metalloproteinases (MMP-2 and MMP-9) messenger RNA as well as miR-181b expression, along with a reduction of metalloproteinase inhibitor 3.", 12, 21)
geneword MMP-2 mirword miR-181b
Conjuncts [MMP-9, MMP-2]
neighbour MMP-9 )
neighbour MMP-9 messenger
Conjunction before fc MMP-2
Conjuncts [MMP-2, MMP-9]
neighbour MMP-2 and
neighbour MMP-2 and
Conjunction before fc MMP-9
Conjuncts [expression, RNA]
neighbour expression ,
neighbour expression along
Conjunction before fc RNA
fc add children for RNA
fc add children for RNA
Conjuncts [RNA, expression]
neighbour RNA as
neighbour RNA as
Conjunction before fc expression
fc add children for expression
Conjunctions
MMP-2 [matrix, metalloproteinases, MMP-2, MMP-9, messenger, RNA, miR-181b, expression]
['MMP-2', 'and', 'MMP-9', ')', 'messenger', 'RNA', 'as', 'well', 'as']
achildren {miR-181b}
achild

{'accept_relation': False,
 'check_results': {'conj': False,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': True,
  'passive': False,
  'negation': False,
  'classification': {'regulation_dir': 'NEU',
   'interaction_dir': 'GENE_MIR',
   'passive': False,
   'reg_detect': 'return',
   'reg_detect_major': 'return'}},
 'tagged_sent': 'Furthermore, treatment of SMCCs with SCF+GM-CSF significantly increased matrix metalloproteinases (<e2>MMP-2</e2> and MMP-9) messenger RNA as well as <e1>miR-181b</e1> expression, along with a reduction of metalloproteinase inhibitor 3.',
 'full_sentence': 'Furthermore, treatment of SMCCs with SCF+GM-CSF significantly increased matrix metalloproteinases (<e2>MMP-2</e2> and MMP-9) messenger RNA as well as <e1>miR-181b</e1> expression, along with a reduction of metalloproteinase inhibitor 3.',
 'entity1': {'entity_type': 'gene',
  'entity_type_token': 'e2',
  'entity_location': (99, 104),
  'entity_text': 'MMP-2'},
 'entity2': {'entity_t

In [24]:
relRes = relChecker.check_sentence(allSents["18523997.2.6"]
                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": (126,134)} #mirna
                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": (65,70)} #gene
                                , fix_special_chars=False
                                , relClassifier=relClassifier.classify, verbose=True
                                )
relRes

add special case USP25
add special case miR-125b
USP25 11 miR-125b-2 26
(u"We found frequent downregulation of two coding genes, SAMSN1 and USP25, as well as of three miRNA genes, miR-99a, let-7c, and miR-125b-2, which reside in the commonly deleted region in human lung cancer.", 11, 26)
geneword USP25 mirword miR-125b-2
Conjuncts [USP25, SAMSN1]
neighbour USP25 ,
neighbour USP25 as
Conjunction before fc SAMSN1
Conjuncts [SAMSN1, USP25]
neighbour SAMSN1 and
neighbour SAMSN1 and
Conjunction before fc USP25
Conjuncts [let-7c, miR-125b-2, genes]
neighbour let-7c ,
neighbour let-7c and
neighbour miR-125b-2 ,
neighbour miR-125b-2 which
Conjunction before fc genes
fc add children for genes
fc add children for genes
fc add children for genes
Conjuncts [genes, miR-125b-2, let-7c]
neighbour genes ,
neighbour genes miR-99a
neighbour miR-125b-2 ,
neighbour miR-125b-2 which
Conjunction before fc let-7c
Conjuncts [genes, let-7c, miR-125b-2]
neighbour genes ,
neighbour genes miR-99a
neighbour let-7c

{'accept_relation': False,
 'check_results': {'conj': False,
  'sdp': True,
  'compartment': True,
  'context': True,
  'entity': False,
  'passive': False,
  'negation': False,
  'classification': {'regulation_dir': 'NEU',
   'interaction_dir': 'GENE_MIR',
   'passive': False,
   'reg_detect': 'return',
   'reg_detect_major': 'return'}},
 'tagged_sent': 'We found frequent downregulation of two coding genes, SAMSN1 and <e2>USP25</e2>, as well as of three miRNA genes, miR-99a, let-7c, and <e1>miR-125b</e1>-2, which reside in the commonly deleted region in human lung cancer.',
 'full_sentence': 'We found frequent downregulation of two coding genes, SAMSN1 and <e2>USP25</e2>, as well as of three miRNA genes, miR-99a, let-7c, and <e1>miR-125b</e1>-2, which reside in the commonly deleted region in human lung cancer.',
 'entity1': {'entity_type': 'gene',
  'entity_type_token': 'e2',
  'entity_location': (65, 70),
  'entity_text': 'USP25'},
 'entity2': {'entity_type': 'mirna',
  'entity_type_